In [1]:
import os
import black
import jupyter_black
from collections import Counter
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import xml.etree.ElementTree as ET
import pandas as pd
import nltk
import json
nltk.download("stopwords")
nltk.download("punkt")
jupyter_black.load(
    lab=False,
    line_length=80,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
DEBUG:jupyter_black:config: {'line_length': 80, 'target_versions': {<TargetVersion.PY310: 10>}}


In [2]:
csv_file_path = r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\DataSets\open-dataset-for-sentiment-analysis-master\betsentiment-ES-tweets-sentiment-worldcup.csv"


# Function to extract sentiment scores from the json str
def extract_sentiment_scores(json_str):
    sentiment_data = json.loads(json_str)
    return (
        sentiment_data["Neutral"],
        sentiment_data["Negative"],
        sentiment_data["Positive"],
        sentiment_data["Mixed"],
    )


# Define data types
dtype_dict = {
    "tweet_date_created": str,
    "tweet_id": int,
    "tweet_text": str,
    "language": str,
    "sentiment": str,
}

try:
    df = pd.read_csv(csv_file_path, encoding="utf-8", dtype=dtype_dict)
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_path, encoding="latin-1", dtype=dtype_dict)

# Apply the function to the "sentiment_score" column to extract the individual scores
sentiment_scores_list = df["sentiment_score"].map(extract_sentiment_scores)

# Create a new DataFrame with sentiment scores as separate columns
sentiment_scores_df = pd.DataFrame(
    sentiment_scores_list.tolist(),
    columns=["Neutral", "Negative", "Positive", "Mixed"],
)

# Concatenate the sentiment scores DataFrame with the original DataFrame
df = pd.concat([df, sentiment_scores_df], axis=1)

# Drop the original "sentiment_score" column
df.drop("sentiment_score", axis=1, inplace=True)

# Display the resulting DataFrame
print(df)

                 tweet_date_created    tweet_id  \
0               2018-06-17T09:47:48  -136978431   
1        2018-06-28T06:00:16.360000 -1076408320   
2               2018-06-07T22:07:43  1008168960   
3               2018-05-31T21:02:10 -1550348286   
4               2018-06-26T11:02:06 -1604923392   
...                             ...         ...   
1072456         2018-06-17T14:12:56 -1932234751   
1072457  2018-06-17T14:12:56.857000 -1932234751   
1072458         2018-05-14T01:54:14  -677941248   
1072459         2018-06-23T15:22:55 -1139363840   
1072460         2018-06-15T03:17:48  -870973439   

                                                                                                                                                                                                                                                                                                            tweet_text  \
0                                                                        

In [3]:
df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,Neutral,Negative,Positive,Mixed
0,2018-06-17T09:47:48,-136978431,@JoseAMeadeK @miseleccionmx A pero si estás en todo menos en tu campaña y todavía te preguntas porque vas en tercer lugar,es,NEUTRAL,0.597391,0.205260,0.085440,0.111909
1,2018-06-28T06:00:16.360000,-1076408320,"#MarioPereyraDT\n""Tenemos que jugarle a Francia con el mismo equipo"" https://t.co/15ccsyMtOi \n\n#SomosArgentina #Cadena3Mundial https://t.co/aNUZMjDkaR",es,NEUTRAL,0.878757,0.058990,0.049931,0.012322
2,2018-06-07T22:07:43,1008168960,"@miseleccionmx No me pidas eso mi selección, sí a eso voy a los estadios a sacar el FUA.",es,NEGATIVE,0.315568,0.556793,0.056737,0.070902
3,2018-05-31T21:02:10,-1550348286,"Si llega a ser la despedida, no será la mejor...\nPero casi que quiero obligar a todos a mirar 4 años atrás y ver todo lo que, atrás de este tipo, lograron jugadores, dirigentes y todos los que forman parte de @Argentina \nGracias por hacernos mejores (y felices) personas????? https://t.co/amTgw75rdq",es,POSITIVE,0.244512,0.005207,0.686508,0.063773
4,2018-06-26T11:02:06,-1604923392,No se les olvide que nuestro trabajo es construir la dictadura del proletariado.\n#VamosColombia,es,NEUTRAL,0.424387,0.418878,0.085919,0.070817
...,...,...,...,...,...,...,...,...,...
1072456,2018-06-17T14:12:56,-1932234751,Se llegó el dia @miseleccionmx pongan el nombre de México en alto! \nTodos estamos con ustedes.???\n#NadaNosDetiene,es,NEUTRAL,0.394735,0.376690,0.160512,0.068063
1072457,2018-06-17T14:12:56.857000,-1932234751,Se llegó el dia @miseleccionmx pongan el nombre de México en alto! \nTodos estamos con ustedes.???\n#NadaNosDetiene,es,NEUTRAL,0.394735,0.376690,0.160512,0.068063
1072458,2018-05-14T01:54:14,-677941248,"@miseleccionmx @ClubAmerica Oribe Peralta no merece ser de la selección nacional, jugador no es, es ratero por solo cobrar nomina sin rendimiento alguno.",es,NEGATIVE,0.286460,0.603919,0.054673,0.054948
1072459,2018-06-23T15:22:55,-1139363840,mi hermano cree q por hacerme sonidos raros cada que sale un coreano me voy a molestar\n#ElMariachiHaráBailarACorea #COREAndoAlaFinal\n#MexicoVsCorea \n#NadaNosDetiene\n#VamosMéxico,es,NEGATIVE,0.166471,0.757941,0.028321,0.047267


In [4]:
# Step 1: Read the stopwords from the .txt file and store them in a list
custom_stopwords_file = (
    r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\utilities\stopwords.txt"
)
with open(custom_stopwords_file, "r", encoding="utf-8") as file:
    custom_stopwords_list = [line.strip() for line in file]

# Step 2: Create an empty set to hold the stopwords
custom_stopwords_set = set()

# Step 3: Add the stopwords from the list to the set
custom_stopwords_set.update(custom_stopwords_list)

In [5]:
# Pre process removing stopwords\
df_01_wordsfreq = df
stop_words = custom_stopwords_set
# convert the column to string type
df_01_wordsfreq["tweet_text"] = df_01_wordsfreq["tweet_text"].astype(str)
df_01_wordsfreq["tweet_text"] = df_01_wordsfreq["tweet_text"].apply(
    lambda x: " ".join(
        [
            word.lower()
            for word in x.split()
            if word.lower() not in stop_words and word.isalpha()
        ]
    )
)

In [ ]:
df_01_wordsfreq["tweet_text"] = df_01_wordsfreq["tweet_text"].apply(
    lambda x: nltk.word_tokenize(x)
)

word_list = [word for words in df_01_wordsfreq["tweet_text"] for word in words]

word_freq = Counter(word_list)

top_words = word_freq.most_common(20)

In [ ]:
# Get the top 10 most common words
top_words = dict(word_freq.most_common(10))
# Create a bar chart of the top 10 most common words
plt.grid(axis="y")
plt.xticks(rotation=45)
plt.xlabel("Words", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.title("Palabras más frecuentes", fontsize=16)
style.use("ggplot")
plt.bar(
    list(top_words.keys()), list(top_words.values()), color=sns.color_palette()
)

In [ ]:
# 1. Count of Each Sentiment
sentiment_counts = df["sentiment"].value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(
    x=sentiment_counts.index, y=sentiment_counts.values, palette="viridis"
)
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.title("Count of Each Sentiment")
plt.show()

In [ ]:
# Box plot
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=df[["Neutral", "Negative", "Positive", "Mixed"]], palette="Set3"
)
plt.xlabel("Sentiment")
plt.ylabel("Sentiment Score")
plt.title("Distribución de los Scores")
plt.show()

In [ ]:
# Violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(
    data=df[["Neutral", "Negative", "Positive", "Mixed"]], palette="Set2"
)
plt.xlabel("Sentiment")
plt.ylabel("Sentiment Score")
plt.title("Distribution of Sentiment Scores")
plt.show()

In [ ]:
# Convert 'tweet_date_created' column to pandas datetime type
df['tweet_date_created'] = pd.to_datetime(df['tweet_date_created'])

# Set the 'tweet_date_created' column as the DataFrame index
df.set_index('tweet_date_created', inplace=True)

# Resample the DataFrame to a specific time frequency (e.g., daily, weekly, monthly)
# For example, to resample to daily frequency, use 'D'
# For weekly, use 'W', and for monthly, use 'M'
time_frequency = 'D'
df_resampled = df.resample(time_frequency).mean()

# Plotting the time series of sentiment scores
plt.figure(figsize=(10, 6))
plt.plot(df_resampled['Neutral'], label='Neutral', linestyle='-')
plt.plot(df_resampled['Negative'], label='Negative', linestyle='--')
plt.plot(df_resampled['Positive'], label='Positive', linestyle='-.')
plt.plot(df_resampled['Mixed'], label='Mixed', linestyle=':')
plt.xlabel('Date')
plt.ylabel('Sentiment Score')
plt.title('Time Series Analysis of Sentiment Scores')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Resample the DataFrame to daily frequency and calculate the count of tweets for each day
daily_tweet_count = df.resample("D").size()

# Plotting the daily tweet count
plt.figure(figsize=(10, 6))
plt.plot(
    daily_tweet_count.index, daily_tweet_count.values, marker="o", linestyle="-"
)
plt.xlabel("Date")
plt.ylabel("Tweet Count")
plt.title("Daily Tweet Count")
plt.grid(True)
plt.show()

In [ ]:
df